# Chapter 16. Dealing DataFrame(Merging, Concatenating)

## 여러 개의 DataFrame 합치기
 - concatenating(연결): 단순히 하나의 DataFrame에 다른 DataFrame을 연속적으로 붙이는 방식
 - merging(병합): 두 DataFrame에 공통적으로 포함되어 있는 하나의 열을 기준으로 삼아, 해당 열의 값이 동일한 두 개의 행들을 하나의 행으로 합치는 방식

In [2]:
import pandas as pd
import numpy as np 

In [3]:
df1 = pd.DataFrame({"key": list("bbacaab"),
                    "data1": range(7)})
df2 = pd.DataFrame({"key": list("abd"),
                    "data2": range(3)})

In [4]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [6]:
df1.merge(df2,on="key") # inner join 이므로 key = c 는 사라지게 된다. 

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [7]:
pd.merge(df1,df2,on="key")

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


 - 관계형 데이터베이스(RDBMS)를 다뤄보신 분이라면, 위에서 pd.merge(df1, df2, on="key")를 실행한 결과가 'inner join' 연산을 수행한 결과임을 확인하실 수 있습니다.
 - how 인자의 값을 달리 입럭하면, 방금 전과는 다른 조합 방식에 따라 새로운 행들을 생성합니다.
 - left, right, inner, outer 모두 가능.

In [11]:
pd.merge(df1, df2, on="key", how="outer")

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [12]:
pd.merge(df1, df2, on="key", how="left")

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [9]:
df3 = pd.DataFrame({"lkey": list("bbacaab"),
                    "data1": range(7)})
df4 = pd.DataFrame({"rkey": list("abd"),
                    "data2": range(3)})

 - pd.merge() 함수를 사용할 때, left_on="lkey", right_on="rkey"를 명시하여 첫 번째 DataFrame의 기준 열과 두 번째 DataFrame의 기준 열을 직접 명시하면 됩니다.

In [13]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


## 인덱스 기준 merging

merging의 대상이 되는 DataFrame 상에서 인덱스를 기준으로 삼고자 하는 경우에도 앞에서 설명한 방법과 크게 다르지 않습니다. <br>
새로운 두 DataFrame을 생성합니다.

In [14]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [20]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [21]:
right1

,group_val
a,3.5
b,7.0


 - left1에서는 "key" 열을, right1에서는 인덱스를 기준으로 merging을 수행하고자 할 경우

In [22]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


 - 첫 번째 인자로 명시된 left1의 경우 "key" 열을 기준으로, 두 번째 인자로 명시된 right1의 경우 인덱스를 기준으로 merging을 하라는 것입니다.
  - 만약 두 DataFrame의 순서가 뒤바뀌는 경우, right_index=True 대신 left_index=True, left_on="key" 대신 right_on="key" 인자를 넣어주시면 됩니다.

## 두 DataFrame 모두 인덱스 기준으로 mergingdmf 수행하고자 할 경우

In [23]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], 
                     index=['a', 'c', 'e'], columns=['Seoul', 'Incheon'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'], columns=['Daegu', 'Ulsan'])

In [25]:
left2

,Seoul,Incheon
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [27]:
right2

,Daegu,Ulsan
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [29]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Seoul,Incheon,Daegu,Ulsan
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


## Concatenating DataFrame

Concatenating은 쉽게 말해서 하나의 DataFrame에 다른 DataFrame을 행 방향 또는 열 방향으로 단순 연결하는 것입니다. 
<br>다음과 같은 3개의 Series를 생성합니다.

In [30]:
s1 = pd.Series([0, 1], index=["a", "b"])
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"])
s3 = pd.Series([5, 6], index=["f", "g"])

In [32]:
s1

a    0
b    1
dtype: int64

In [33]:
s2

c    2
d    3
e    4
dtype: int64

In [34]:
s3

f    5
g    6
dtype: int64

 - pd.concat() 함수를 사용하여 3개의 Series를 연결하면, 단순히 1차원적으로 순서대로 연결된 새로운 Series가 생성됩니다.

In [36]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

 - pd.concat() 함수의 axis=1 인자를 명시하면, Series가 2차원적으로 연결되면서 DataFrame이 생성됩니다.

In [38]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


 - s1의 인덱스를 일부 포함하는 새로운 s4 Series를 하나 생성하고, 다시 concatenating을 진행하면, s4와 s1이 공유하는 인덱스인 'a'와 'b'의 경우 같은 행 상에서 연결된 것을 확인할 수 있습니다.

In [40]:
s4 = pd.concat([s1 * 5, s3])

In [45]:
print(s4,"\n",s1)

a    0
b    5
f    5
g    6
dtype: int64 
 a    0
b    1
dtype: int64


In [46]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


- concatenating을 진행할 때, 연결된 DataFrame에 컬럼을 다음과 같이 명시해줄 수도 있습니다.

In [48]:
pd.concat([s1, s2, s3], axis=1, keys=["one", "two", "three"])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


- 두 개의 DataFrame에 대해서도 완전히 동일한 방식으로 concatenating이 가능합니다.

In [50]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), 
                   index=['a', 'b', 'c'], columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), 
                   index=['a', 'c'], columns=['three', 'four'])
pd.concat([df1, df2], axis=1)

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


 - 행 방향으로 concatenating을 수행할 때 기존 DataFrame의 인덱스가 유지되었는데, pd.concat() 함수에서 ignore_index=True 인자를 넣어주면 기존의 인덱스를 무시하고 새로운 인덱스를 붙여줍니다.

In [51]:
df3 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df4 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
pd.concat([df3, df4], ignore_index=True)

,a,b,c,d
0,-0.867553,0.334805,1.030291,1.229187
1,-0.099104,0.838899,-0.477750,0.299059
2,0.792631,-0.062884,-0.141750,-0.802274
3,1.354258,0.225077,NaN,-1.370276
4,0.101107,0.277331,NaN,0.769083


In [52]:
pd.concat([df3, df4])

,a,b,c,d
0,-0.867553,0.334805,1.030291,1.229187
1,-0.099104,0.838899,-0.477750,0.299059
2,0.792631,-0.062884,-0.141750,-0.802274
0,1.354258,0.225077,NaN,-1.370276
1,0.101107,0.277331,NaN,0.769083
